## Modelo Preditivo: XGBoost (eXtreme Gradient Boosting)

O modelo XGBoost é um dos algoritmos que utilizamos, baseado em árvores de decisão e utilizando a estrutura de Gradient Boosting. Ele funciona combinando árvores de decisão em sequência para corrigir possíveis erros dos modelos anteriores, o que aumenta a precisão das previsões.

### Instruções para uso local ou remoto (Google Colab ou VScode)

Aqui deixaremos brevemente um passo a passo para que você usuário seja capaz de executar o código localmente ou remotamente pelo seu google drive, podendo escolher a forma mais viável para seu uso e conhecimento.

##### Google Colab

1. Faça o upload do seu arquivo .ipynb para o Google Drive.
2. Abra o Google Colab em seu navegador.
3. Clique em "Arquivo" no menu superior e selecione "Abrir notebook".
4. Na guia "Upload", clique em "Procurar" e selecione o arquivo .ipynb que você enviou para o Google Drive.
5. Após selecionar o arquivo, clique em "Abrir".
6. Aguarde o carregamento do notebook no Google Colab.
7. Agora que você carregou o notebook no Google Colab, você pode fazer as alterações necessárias nos arquivos e caminhos para se adequar ao seu ambiente específico.

No notebook, a célula seguinte à essa contém as leituras dos arquivos CSV com o caminho do drive do criador desse notebook.
Comente as linhas que fazem referência aos arquivos locais e descomente as linhas que fazem referência ao Google Drive. Por exemplo:

- descomente as linhas que começam com # from google.colab import drive;
- comente as linhas que começam com tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv").

Certifique-se de que os arquivos CSV estejam localizados no diretório correto em seu ambiente virtual. Por exemplo, se você tiver uma pasta chamada "data" no mesmo diretório do notebook, coloque os arquivos CSV nessa pasta e ajuste seus nomes. Possivelmente os arquivos vão seguir o seguinte padrão, mesmo no seu drive:

tabela_Meta = pd.read_csv("/content/drive/MyDrive/NomeDaPastaDosArquivos/arquivo.csv)

Salve as alterações no notebook.
Agora você pode executar as células do notebook no Google Colab e as alterações nos arquivos e caminhos serão aplicadas ao seu ambiente do colab. Lembre-se de que você não precisará ter as bibliotecas necessárias instaladas em seu ambiente local para executar o código corretamente, dado que ao executar pela ferramenta do google essas dependências estarão aplicadas por padrão

##### VScode

1. Coloque os arquivos CSV dentro da pasta "data" desse notebook

No notebook, a célula seguinte à essa contém as leituras dos arquivos CSV com o caminho do drive do criador desse notebook.
Comente as linhas que fazem referência aos arquivos locais e descomente as linhas que fazem referência ao Google Drive. Por exemplo:

- descomente as linhas que começam com # from google.colab import drive;
- comente as linhas que começam com tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv").

Certifique-se de que os arquivos CSV estejam localizados no diretório correto em seu ambiente virtual. Por exemplo, se você tiver uma pasta chamada "data" no mesmo diretório do notebook, coloque os arquivos CSV nessa pasta e ajuste seus nomes. Possivelmente os arquivos vão seguir o seguinte padrão, mesmo no seu drive:

tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv")

Salve as alterações no notebook.
Agora você pode executar as células do notebook no VScode e as alterações nos arquivos e caminhos serão aplicadas ao seu ambiente do colab. Lembre-se de que você precisará ter as bibliotecas necessárias instaladas em seu ambiente local para executar o código corretamente:

Para fazer a instalação, basta abrir o terminal integrado e inserir o seguinte:

In [ ]:
#pip install pandas numpy matplotlib scikit-learn shap

### Execução do Modelo

O primeiro passo para realizarmos nossas previsões, é importar todas as bibliotecas que serão usadas na geração do modelo.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import shap

Abaixo, usamos os arquivos na pasta "data" com os nomes descritos, mas lembre-se de mudar conforme necessidade;

In [5]:
# # pegando arquivos csv do drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Lendo os arquivos CSV
# tabela_Meta = pd.read_csv("/content/drive/MyDrive/Primeiro Ano/Módulo 3 - Modelo Preditivo Gazeta/Base de Dados Limpas/BASE INTELI_META-limpo.csv")
# tabela_Agosto = pd.read_csv("/content/drive/MyDrive/Primeiro Ano/Módulo 3 - Modelo Preditivo Gazeta/Base de Dados Limpas/tratada_BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv")

# para realizar o processo localmente, descomente as linhas abaixo e comente as linhas acima.
tabela_Meta = pd.read_csv("../data/dados_tratados/BASE INTELI_META-limpo.csv")
tabela_Agosto = pd.read_csv("../data/dados_tratados/tratada_BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv")

### Seleção das features
Com os arquivos em mãos, vamos definir os valores máximos no formato de exibição e agrupar as colunas, a fim de somar os valores numéricos conforme as exigências do parceiro. Ao final, a saída exibirá uma representação da tabela filtrada com as características utilizadas.

In [ ]:
tabela_Agosto.head()

In [ ]:
# Definindo formato de exibição
# pd.set_option('display.float_format', '{:.0f}'.format)

# Agrupar e somar 'Vl Liquido Final' e 'Outra Coluna'
tabela_agosto_segmento = tabela_Agosto.groupby(['Ano', 'Mês','Veiculo', 'Origem', 'Segmento','% Ating. Meta']).agg({
    'Vl Liquido Final': 'sum',
}).reset_index()

tabela_agosto_segmento.head()


Portanto, de acordo com a tabela filtrada, as features utilizadas são as:
- **Ano**: Permite avaliar o desempenho da receita ao longo do ano.
- **Mês**: Indica em qual mês a receita teve maior impacto.
- **Veículo**: Analisa o impacto de cada veículo de venda na receita final.
- **Origem**: Identifica a localidade da venda para verificar a região de maior impacto na receita.
- **Segmento**: Permite filtrar receitas por setor econômico de forma mais detalhada (segmento).
- **% Ating. Meta**: Identifica a porcentagem da meta alcançada durante o mês
- **Vl. Líquido**: Utilizado para prever a receita final, pois os ganhos estão relacionados a esse valor.

### Adaptação das features
Daqui em diante, iremos trabalhar mais com ajustes das colunas para que o modelo seja capaz de compreender as informações.
 Abaixo, as colunas categóricas (aquelas que são textos) serão transformadas em numéricas, de modo a pouco comprometer o desempenho do modelo.

In [8]:
categorical_columns = ['Origem','Veiculo', 'Segmento']
X = tabela_agosto_segmento.drop('Vl Liquido Final', axis=1)
y = tabela_agosto_segmento['Vl Liquido Final']

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le


### Treinamento do modelo e definição de hiperparâmetros

Após, faremos a divisão das amostras de treinamento e teste do modelo, e depois o ajuste do modelo com o grupo de treino, sendo que 70% dos dados são destinados para o treino e 30% para o teste.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

Agora, iremos iniciar o treinamento do nosso modelo XGBoost, definindo os melhores parâmetros e treinando o modelo para realizar as previsões


In [10]:
xgb = XGBRegressor(n_estimators=81, max_depth=7, learning_rate=0.1, random_state=5, subsample= 1, min_child_weight=0.5, colsample_bytree= 1, reg_alpha= 0, reg_lambda= 2)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)


Analisando cada hiperparâmetro:

- n_estimators: Define o número de árvores no modelo, ou seja, quantas iterações o algoritmo realizará. Um valor mais alto pode melhorar a precisão, mas pode aumentar o tempo de treinamento e o risco de overfitting.

- max_depth: Define a profundidade máxima de cada árvore. Profundidades maiores permitem que o modelo capture padrões mais complexos, mas também aumentam o risco de overfitting. Um valor de 7 permite um bom equilíbrio entre complexidade e generalização.

- learning_rate: Controla a taxa de aprendizado do modelo, ou seja, o quanto cada árvore contribui para o modelo final. Um valor mais baixo faz com que o modelo aprenda de maneira mais gradual, potencialmente melhorando a precisão, mas exigindo mais iterações.

- random_state: Garante a reprodutibilidade dos resultados, fixando a semente do gerador de números aleatórios. Isso permite que os mesmos dados e parâmetros produzam sempre o mesmo modelo.

- subsample: Define a fração dos dados usados em cada árvore. Um valor de 1 significa que todas as amostras serão utilizadas para construir cada árvore. Valores menores podem ajudar a reduzir o overfitting.

- min_child_weight: Determina o peso mínimo necessário para dividir um nó em uma árvore. Valores menores permitem a criação de nós com menos amostras, o que pode aumentar a flexibilidade, mas aumentar o risco de overfitting.

- colsample_bytree: Especifica a fração de recursos (features) que será usada em cada árvore. Um valor de 1 significa que todas as features serão usadas. Reduzir esse valor pode evitar o overfitting.

- reg_alpha: Controla a regularização L1, que adiciona uma penalidade ao modelo para evitar overfitting. Um valor de 0 significa que essa regularização não está sendo aplicada.

- reg_lambda: Controla a regularização L2, que também adiciona uma penalidade ao modelo, mas de maneira diferente da L1. Um valor maior impõe uma penalização mais forte, ajudando a prevenir overfitting.

Então, a partir dos valores obtidos anteriormente, vamos aplicar o Grid Search a fim de encontrar os melhores hiperparâmetros para este modelo, em seguida ajustaremos a implementação com os novos valores encontrados no treinamento do modelo.

### Grid Search
O Grid Search é uma técnica utilizada para encontrar os melhores hiperparâmetros de um modelo. Ele explora sistematicamente diversas combinações de parâmetros e retorna aquela que proporciona o melhor desempenho, com base em uma métrica de avaliação definida. Essa abordagem otimiza a performance do modelo e ajuda a evitar problemas como overfitting, garantindo um ajuste mais eficiente aos dados.

In [11]:
# param_grid = {
#     'n_estimators': list(range(40, 100)),
#     'learning_rate': [0.01, 0.1, 0.2],
#     'max_depth': [3, 4, 5, 6, 7],
#     'subsample': [0.1, 0.3, 0.4, 0.7, 1],
#     'min_child_weight': [0.5, 1, 2, 3, 4, 5],
#     'colsample_bytree': [0.7, 0.8, 1], 
#     'reg_alpha': [0, 0.01, 0.1],        
#     'reg_lambda': [1, 1.5, 2]           
# }

# # Configurando o Grid Search com validação cruzada
# grid_search = GridSearchCV(
#     estimator=xgb,
#     param_grid=param_grid,
#     scoring='neg_mean_absolute_error',  # Métrica de avaliação
#     cv=3,                               # Número de folds para validação cruzada
#     verbose=1,
#     n_jobs=-1                           # Usa todos os núcleos disponíveis
# )

# # Ajustando o Grid Search ao conjunto de dados
# grid_search.fit(X_train, y_train)

# print("Melhores Parâmetros:", grid_search.best_params_)
# print("Melhor Score:", -grid_search.best_score_)

De acordo com o output da célula acima podemos ver que os melhores hiperparametros são:
-  ('colsample_bytree', 1),
- ('learning_rate', 0.1),
- ('max_depth', 7),
- ('min_child_weight', 0.5),
- ('n_estimators', 81),
- ('reg_alpha', 0),
- ('reg_lambda', 2),
- ('subsample', 1)
E o melhor score obtido para a média absoluta de erro é de 14034.448486908901

Obtendo esses valores, nós alteramos em nosso modelo e obtemos o resultado das métricas abaixo.

### Métricas

Depois de realizar as previsões, vamos verificar qual o valor de cada métrica para conferir como o modelo está atuando e como está prevendo os valores.

#### Erro Médio Absoluto (MAE - do inglês *Mean Absoluto Error*)
O MAE é uma métrica utilizada para avaliar a qualidade de modelos de regressão, calculando a média das diferenças absolutas entre os valores reais e os valores preditos. Quanto menor o valor do MAE, melhor o desempenho do modelo. Abaixo, apresentamos o cálculo desse indicador no modelo do grupo:<br>

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

#### Raiz do Erro Quadrático Médio (RMSE - do inglês *Root Mean Squared Error*)

O RMSE (Root Mean Squared Error) é uma métrica de avaliação utilizada em modelos de regressão que, assim como o MSE (Mean Squared Error), mede a diferença entre os valores reais e os valores previstos. No RMSE, essa diferença é elevada ao quadrado, o que elimina os sinais negativos e dá mais peso a grandes erros. A diferença principal está no fato de que, ao final, é aplicada a raiz quadrada no valor resultante, tornando a métrica mais interpretável, pois os erros são trazidos de volta à mesma escala das variáveis originais. Portanto, quanto menor o RMSE, melhor o desempenho do modelo.

In [ ]:
rmse = mean_squared_error(y_test, y_pred)
print(f"Root Mean Squared Error: {sqrt(rmse)}")

#### Coeficiente de Determinação (R²)
O Coeficiente de Determinação R² é uma métrica que representa o percentual da variância dos dados previstos, ou seja, o quão explicativo é o modelo em relação aos dados de acordo com o quão distante esses valores estão do valor central (médio). Uma vez que a fórmula do R² considera a subtração desta conta por 1, ou seja, um modelo entre 0,8 e 0,9 é um bom indicador. Este, no entanto, não é suficiente para ter uma noção geral da performace do modelo, dependendo de outras métricas (como o MAE e o MSE). Abaixo, apresentamos o cálculo desse indicador no modelo do grupo:

In [ ]:
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

#### Erro Médio Absoluto Percentual (MAPE - do inglês Mean Absolute Percentage Error)

O MAPE é uma métrica utilizada para avaliar a precisão de modelos de regressão, calculando a média dos erros percentuais absolutos entre os valores reais e os valores preditos. Quanto menor o valor do MAPE, melhor o desempenho do modelo. Abaixo, apresentamos o cálculo desse indicador no modelo do grupo:

In [ ]:

mape = mean_absolute_percentage_error(y_test, y_pred)
print (f"MAPE: {mape}")

### Conclusões a partir das métricas

O modelo XGBoost está apresentando um desempenho excelente, com resultados semelhantes ao Random Forest, porém com um R² inferior, indicando uma menor tendência ao overfitting. Além disso, todas as métricas mostram valores baixos, o que é um ótimo indicativo de que o modelo está funcionando de maneira eficiente e conseguindo prever as receitas com precisão.

A seguir, vamos analisar os resultados visualmente:

O gráfico compara os valores reais com as previsões do modelo. A linha roxa representa as previsões geradas pelo modelo, enquanto os pontos vermelhos correspondem aos valores reais. Quanto mais próximos os pontos estiverem da linha, melhor a qualidade das previsões.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='red', alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='purple', lw=3)  # Linha de identidade
plt.xlabel('Valores Reais')
plt.ylabel('Previsões')
plt.title('Comparação entre Valores Reais e Previsões - Regressão Polinomial')
plt.grid(True)
plt.show()

## Explicabilidade
A explicabilidade de um modelo preditivo refere-se à capacidade de compreender e interpretar como um modelo toma suas decisões ou faz previsões. Isso é particularmente importante em modelos complexos onde o processo de decisão não é facilmente visível para os seres humanos. Logo abaixo, vamos iniciar a explicabilidade do modelo para conferir como as previsões foram realizadas.

In [ ]:
# Crie um objeto explainer SHAP
explainer = shap.Explainer(xgb, X_train)

# Calcule os valores SHAP para as previsões
shap_values = explainer(X_test)


#### Summary Plot
Esse gráfico de resumo mostra a importância global das variáveis no modelo, bem como a distribuição dos valores SHAP (impacto no resultado do modelo) para cada variável. Ele mostra como cada variável impacta globalmente o modelo e sua distribuição nos dados.

In [ ]:
# Visualize a importância de recursos e sua distribuição.
shap.summary_plot(shap_values, X_test)

- Eixo Y: Lista as variáveis/features do modelo, como "Segmento", "Veículo", "Origem", etc.
- Eixo X: Representa os valores SHAP, que indicam o impacto da variável no resultado do modelo. Valores positivos aumentam a previsão, enquanto valores negativos a reduzem.
- Pontos: Cada ponto representa uma observação (amostra) dos seus dados.
A cor do ponto indica o valor da variável: azul para valores baixos, vermelho para valores altos.
A posição horizontal do ponto indica o impacto que aquela observação teve no resultado. Quanto mais à direita, maior o impacto positivo na previsão; quanto mais à esquerda, maior o impacto negativo.

A variável Segmento tem um impacto maior em geral, com muitos pontos azuis e vermelhos espalhados. Isso indica que, dependendo do valor da variável (seja alto ou baixo), ela pode ter um impacto positivo ou negativo nas previsões.
% Ating. Meta também tem uma distribuição de valores SHAP, mas tem menos impacto em comparação com "Segmento" e "Veículo".

#### Waterfall Plot
Este gráfico explica a previsão individual de um dado ponto de teste. Ele decompõe o valor predito em contribuições individuais de cada variável. O valor final predito é a soma da base do modelo com as contribuições de cada variável. Ou seja, ele explica como cada variável contribui para a previsão de um ponto específico, mostrando o impacto individual de cada uma na previsão final.

In [ ]:

# Mostra como cada recurso impactou uma previsão específica.
shap.waterfall_plot(shap_values[0])

- f(x): O valor predito para essa observação específica (no caso, aproximadamente -2281.90).
- Base value: O valor de referência do modelo, antes de considerar os efeitos das variáveis (aproximadamente 46879.08).
- Barras Azuis/Vermelhas:
Barras azuis indicam que a variável diminuiu a previsão em relação ao valor base.
Barras vermelhas indicam que a variável aumentou a previsão.
Cada barra mostra o valor da variável e seu impacto específico na previsão.

A variável Segmento (com valor 7) reduziu a previsão em -29492.33.
A variável Veículo (com valor 13) também teve um impacto negativo, reduzindo a previsão em -25174.16.
Por outro lado, variáveis como % Ating. Meta (valor 1.251) aumentaram a previsão em +7042.44.